In [1]:
import sys
sys.path.append('//anaconda/lib/python2.7/site-packages/')
from IPython.display import display
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.cluster import FeatureAgglomeration
from sklearn import svm
import matplotlib
import matplotlib.pyplot as plt
import nltk
from datetime import datetime as dtdm
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, euclidean
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from textblob import TextBlob
from datetime import datetime as dt

%matplotlib inline

## Sentiment Analysis

We believed that using sentiment analysis of Airbnb reviews would enhance the color of the dataset and improve our accuracy of pricing prediction overall.  In order to perform sentiment analysis, we first had to clean the dataset of reviews for each listing.  This proved to be quite involved because many listings had commas in them which caused the read CSV function to become confused and split reviews inappropriately.  After the intense data cleaning, we used a library called ``text blob`` to find out the polarity score of each review.  We aggregated and found the average of the reviews and assigned those scores to each listing ID, which we then incorporated into our final model.

In [2]:
#the dataset was a CSV with comments that had multiple commas
#the CSV wouldn't read properly if you included certain lines
#those lines are dropped below
reviews1 = pd.read_csv("C:/Users/fahmida/Desktop/Rental/airbnb/Reviews/reviews1.csv.gz")
reviews2 = pd.read_csv("C:/Users/fahmida/Desktop/Rental/airbnb/Reviews/reviews2.csv.gz")
reviews3 = pd.read_csv("C:/Users/fahmida/Desktop/Rental/airbnb/Reviews/reviews3.csv.gz")
reviews4 = pd.read_csv("C:/Users/fahmida/Desktop/Rental/airbnb/Reviews/reviews4.csv.gz")
reviews5 = pd.read_csv("C:/Users/fahmida/Desktop/Rental/airbnb/Reviews/reviews5.csv.gz")
reviews6 = pd.read_csv("C:/Users/fahmida/Desktop/Rental/airbnb/Reviews/reviews6.csv.gz")
reviews7 = pd.read_csv("C:/Users/fahmida/Desktop/Rental/airbnb/Reviews/reviews7.csv.gz")

In [3]:
reviews = pd.concat([reviews1,reviews2,reviews3,reviews4,reviews5,reviews6,reviews7])

In [4]:
display(reviews1[:25].T)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
listing_id,1419,1419,1419,1419,1419,1419,1419,8077,8077,8077,...,8077,8077,8077,8077,8077,8077,8077,8077,8077,8077
id,38924112,44791978,45957133,67295154,177702208,179394665,216619832,7830,9941,11995,...,28374,28640,32908,37738,43266,58434,61600,63274,71707,77771
date,2015-07-19,2015-08-29,2015-09-07,2016-03-28,2017-08-03,2017-08-07,2017-12-04,2009-08-20,2009-09-13,2009-09-30,...,2010-03-05,2010-03-07,2010-04-03,2010-04-25,2010-05-17,2010-06-28,2010-07-06,2010-07-11,2010-07-31,2010-08-13
reviewer_id,11308465,9580285,38394721,3515044,13987100,111148735,46323833,12750,30020,36786,...,63186,86496,75771,78792,88526,133129,105890,85743,181730,163172
reviewer_name,Marcela,Marco,Andrea,Shaun,Kate,Melissa,Rachel,Margarita,Sunny,Collin & Nandi,...,Steve & Breanna,Susan,Verena,Quentin,Don,Ruben,Ulikah,Donna,'Bebe',Jennifer
comments,Having the opportunity of arriving to Alexandr...,We have no enough words to describe how beauty...,The listing was exceptional and an even better...,Alexandra's home was amazing and in such a nea...,Beautiful home. Very comfortable and clean. Pe...,Alexandra was such an incredible host during o...,The host canceled this reservation 22 days bef...,We had a lovely time in Toronto! Kathie and La...,"Great hosts, great apartment and we loved stay...",We really enjoyed staying at Kathie and Larry'...,...,We had such a great experience at Kathie and L...,"Wonderful hosts, wonderful room and an amazing...",Kathie and Larry were great hosts that warmly ...,We can't think of any reason to stay anywhere ...,Had an absolutely wonderful time. Larry and Ka...,Had a wonderful time @ Katie&Larry's! They mad...,"WONDERFULL couple, fantasic room. You guys mad...",Larry and Kathy - had a great time. Thanks fo...,"Kathie and Larry are welomeing, intelligent, c...",Kathie and Larry were absolutely tremendous. T...


As you can see above, the commas in the comments confused the CSV reader and caused the data to be read in incorrectly.  You can see this in the rows in which ``reviewer_name`` and ``comments`` have split up strings and the names of the reviewers have been shifted up to ``reviewer_id.``  The next kernels will work to ameliorate this issue, as well as fix the NaNs in the dataset.

In [5]:
reviews['comments_combined'] = ''

reviews = reviews[pd.notnull(reviews.listing_id)&pd.notnull(reviews.id)&pd.notnull(reviews.date)]

In [6]:
display(reviews[:25].T)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
listing_id,1419,1419,1419,1419,1419,1419,1419,8077,8077,8077,...,8077,8077,8077,8077,8077,8077,8077,8077,8077,8077
id,38924112,44791978,45957133,67295154,177702208,179394665,216619832,7830,9941,11995,...,28374,28640,32908,37738,43266,58434,61600,63274,71707,77771
date,2015-07-19,2015-08-29,2015-09-07,2016-03-28,2017-08-03,2017-08-07,2017-12-04,2009-08-20,2009-09-13,2009-09-30,...,2010-03-05,2010-03-07,2010-04-03,2010-04-25,2010-05-17,2010-06-28,2010-07-06,2010-07-11,2010-07-31,2010-08-13
reviewer_id,11308465,9580285,38394721,3515044,13987100,111148735,46323833,12750,30020,36786,...,63186,86496,75771,78792,88526,133129,105890,85743,181730,163172
reviewer_name,Marcela,Marco,Andrea,Shaun,Kate,Melissa,Rachel,Margarita,Sunny,Collin & Nandi,...,Steve & Breanna,Susan,Verena,Quentin,Don,Ruben,Ulikah,Donna,'Bebe',Jennifer
comments,Having the opportunity of arriving to Alexandr...,We have no enough words to describe how beauty...,The listing was exceptional and an even better...,Alexandra's home was amazing and in such a nea...,Beautiful home. Very comfortable and clean. Pe...,Alexandra was such an incredible host during o...,The host canceled this reservation 22 days bef...,We had a lovely time in Toronto! Kathie and La...,"Great hosts, great apartment and we loved stay...",We really enjoyed staying at Kathie and Larry'...,...,We had such a great experience at Kathie and L...,"Wonderful hosts, wonderful room and an amazing...",Kathie and Larry were great hosts that warmly ...,We can't think of any reason to stay anywhere ...,Had an absolutely wonderful time. Larry and Ka...,Had a wonderful time @ Katie&Larry's! They mad...,"WONDERFULL couple, fantasic room. You guys mad...",Larry and Kathy - had a great time. Thanks fo...,"Kathie and Larry are welomeing, intelligent, c...",Kathie and Larry were absolutely tremendous. T...
comments_combined,,,,,,,,,,,...,,,,,,,,,,


In [7]:
nan_reviews = reviews[pd.isnull(reviews.comments)]

In [8]:
nan_reviews.comments_combined = nan_reviews.reviewer_name
nan_reviews.drop(['reviewer_name','comments'],axis=1,inplace=True)
display(nan_reviews.T)

C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,26357,29593,30134,32401,32918,41492,42298,47305,48136,51088,...,404391,404881,406881,407423,408777,409458,409601,410771,410801,411157
listing_id,1727775,1968332,2016823,2359982,2429122,3290328,3339294,3852422,3873403,3987246,...,26061770,26136684,26491528,26588287,26887331,27097530,27157023,27571614,27579367,27786870
id,279749191,405655154,99787824,199685838,177677520,195221087,184384250,241541933,322271721,161325855,...,310528384,306424306,310488849,306432252,311616828,310529332,299285059,307540571,310386216,310446139
date,2018-06-21,2019-01-27,2016-09-05,2017-10-02,2017-08-03,2017-09-18,2017-08-19,2018-03-09,2018-09-12,2017-06-17,...,2018-08-19,2018-08-12,2018-08-19,2018-08-12,2018-08-21,2018-08-19,2018-07-30,2018-08-14,2018-08-19,2018-08-19
reviewer_id,87493392,60489702,50640863,104815329,141167350,97693758,139778678,176834401,194111820,34147306,...,61391196,60905458,138799131,84053320,70551417,153315425,53299717,206302533,208139981,209970112
comments_combined,Andrea,Jacob,Michelle,Ahmad,Edivaldo,Robert,Rafie,Zeeshan,Vanessa,Krish,...,Phillip,Phyllis,Miranda,Madeleine,Jeff,Elaine,Nathan,Andy,宥衣,Shu


In [9]:
is_reviews = reviews[pd.notnull(reviews.comments)]
is_reviews.comments_combined = is_reviews.reviewer_name+is_reviews.comments
is_reviews.drop(['reviewer_name','comments'],axis=1,inplace=True)
display(is_reviews.T)

,0,1,2,3,4,5,6,7,8,9,...,411467,411468,411469,411470,411471,411472,411473,411474,411475,411476
listing_id,1419,1419,1419,1419,1419,1419,1419,8077,8077,8077,...,28217940,28239659,28246537,28255238,28255240,28276210,28283510,28285243,28311165,28336351
id,38924112,44791978,45957133,67295154,177702208,179394665,216619832,7830,9941,11995,...,318250895,318835425,319505413,319889299,318808903,320413340,319884531,319909474,319394240,320304320
date,2015-07-19,2015-08-29,2015-09-07,2016-03-28,2017-08-03,2017-08-07,2017-12-04,2009-08-20,2009-09-13,2009-09-30,...,2018-09-03,2018-09-04,2018-09-06,2018-09-07,2018-09-04,2018-09-08,2018-09-07,2018-09-07,2018-09-06,2018-09-08
reviewer_id,11308465,9580285,38394721,3515044,13987100,111148735,46323833,12750,30020,36786,...,37549231,88277286,19387643,71382892,9803773,214293711,23648824,175245626,49998241,184694345
comments_combined,MarcelaHaving the opportunity of arriving to A...,MarcoWe have no enough words to describe how b...,AndreaThe listing was exceptional and an even ...,ShaunAlexandra's home was amazing and in such ...,KateBeautiful home. Very comfortable and clean...,MelissaAlexandra was such an incredible host d...,RachelThe host canceled this reservation 22 da...,MargaritaWe had a lovely time in Toronto! Kath...,"SunnyGreat hosts, great apartment and we loved...",Collin & NandiWe really enjoyed staying at Kat...,...,EddieGreat place to stay a great location .,LydiaI was their 1st guest ever !!!Amazing fam...,RyanThe perfect spot for a getaway in Toronto!...,MichaelSuper clean! Super tidy! Everything is ...,BradI've stayed in many Airbnb's and this is o...,DanielThe host canceled this reservation 30 da...,JohnMy stay was very nice. I met with Jean's ...,AjayIt’s good but noisy place,JessicaThe host canceled this reservation 9 da...,GabyBill was a wonderful host. He was very hel...


In [10]:
reviews = pd.concat([nan_reviews,is_reviews])
reviews.columns = ['listing_id','id','date','reviewer_id','comments']
reviews = reviews[pd.notnull(reviews.comments)]
display(reviews[225000:225050].T)

,221641,221642,221643,221644,221645,221646,221647,221648,221649,221650,...,221681,221682,221683,221684,221685,221686,221687,221688,221689,221690
listing_id,14691049,14691049,14691049,14691049,14691049,14691049,14691049,14691049,14691049,14691049,...,14691049,14691049,14691049,14691049,14691049,14691049,14691049,14691049,14691049,14691049
id,108552310,110953629,111240370,112295568,113375825,114856942,116029257,117055692,124325657,128130390,...,221418660,222063127,223084373,223734221,226990205,227323022,228612726,229033126,230239823,234023929
date,2016-10-16,2016-10-29,2016-10-30,2016-11-05,2016-11-12,2016-11-20,2016-11-27,2016-12-04,2017-01-01,2017-01-22,...,2017-12-26,2017-12-28,2017-12-31,2018-01-01,2018-01-13,2018-01-14,2018-01-20,2018-01-21,2018-01-27,2018-02-11
reviewer_id,40722277,34014964,97799654,35693079,38675401,37824424,40245827,8716170,27913911,67640368,...,41477510,157294749,83674786,53145015,65060381,69443520,54263855,129334525,161924173,49804268
comments,VirginiaTiruneh's place was a good space for 4...,ShanehI've stayed at a couple Airbnb's downtow...,LaurynHad a great stay! Hosts are easy to get ...,JayceA great and convenient place for a short ...,"TajanaTiruneh's place had a great location, ea...",AmandaSuch a great space!,"Geng HuiAmenities provided were as promised, w...",RyanThe host is hard to reach. They don't repl...,Johnwe wrote already,UzairTiruneh was an amazing host and the place...,...,GustavGreat location. Easy to check in. The sp...,JoelGreat location with McDonald's and LCBO wi...,Masonnice and big room,VinTiruneh’s place is located at the heart of ...,Luis-DanielGreat location! Host very responsive.,DevenBeautiful and clean place well worth the ...,JessicaVery hospitable and helpful.,"ManyaGreat host, nice space! Very central and ...",KaylinGreat stay very spacious!,JordanGreat listing! Helpful and responsive ho...


The data cleaning of the reviews is finally complete.  It is now time to apply sentiment analysis to the cleaned dataset.

In [ ]:
#apply the sentiment analysis to each review
reload(sys)
sys.setdefaultencoding("utf-8")
tok_reviews = reviews.comments.apply(lambda row: TextBlob(row).sentiment)

In [ ]:
reviews['tok_reviews'] = tok_reviews

In [ ]:
reviews.shape

In [ ]:
#create a series of the polarity and subjectivity score of each review
reviews[['polarity', 'subjectivity']] = reviews['tok_reviews'].apply(pd.Series)

In [ ]:
display(reviews[:25].T)

In [ ]:
reviews.to_csv('C:/Users/fahmida/Desktop/Rental/airbnb/Reviews/sentiment_reviews.csv')

In [ ]:
groups = reviews[reviews.listing_id.apply(lambda x: x.isnumeric())]

In [ ]:
#find the average polarity score of each listing and assign it to each listing ID
groups = groups.groupby(['listing_id'])['polarity'].mean()

In [ ]:
groups

In [ ]:
df_dummies.shape

In [ ]:
display(groups[:5].T)

In [ ]:
print(groups.index.dtype)

In [ ]:
print(new_unique[:5])

In [ ]:
#show each listing ID and its polarity score
groups = pd.DataFrame(groups)
int_groups_idx = pd.to_numeric(groups.index)
display(groups)

In [ ]:
groups['idx'] = int_groups_idx

In [ ]:
groups.set_index('idx',inplace=True)

In [ ]:
display(groups)

In [ ]:
df_dummies.set_index('id',inplace=True)

In [ ]:
df_dummies['sentiment'] = 0.0

In [ ]:
#assign polarity scores to entire reviews dataset
n = 0
for i in groups.index:

    df_dummies.loc[i,'sentiment'] = groups.loc[i].polarity
    n += 1
    if (n % 100) == 0:
            print(n)

In [ ]:
#write the final results to a csv
df_dummies.to_csv('C:/Users/fahmida/Desktop/Rental/airbnb/Reviews/time_sentiment.csv')